In [67]:
# system
import sys

In [68]:
# TC
from PyQuantum.TC.Cavity import Cavity
from PyQuantum.TC.Hamiltonian import Hamiltonian

from PyQuantum.TC.WaveFunction import WaveFunction
from PyQuantum.TC.DensityMatrix import *

from PyQuantum.TC.Evolution import *

In [69]:
# Common
from PyQuantum.Common.LoadPackage import *
from PyQuantum.Common.STR import *

from PyQuantum.Common.ext import mkdir
from PyQuantum.Common.Print import *
from PyQuantum.Common.PyPlot import PyPlot3D
# from shutil import copyfile
# from numpy.random import rand

In [70]:
config = load_pkg("config", "/home/alexfmsu/Git/PyQuantum/PyQuantum/TC/config.py")

mkdir(config.path)

In [71]:
cavity = Cavity(n=config.n, wc=config.wc, wa=config.wa, g=config.g)

print("Cavity:", color="green")

print()

cavity.print_n()
cavity.print_wc()
cavity.print_wa()
cavity.print_g()

print("T:", config.T)
print("nt:", config.nt)
print("dt:", config.dt)

print()

Cavity:
 n: 2

wc: 21.506 GHz

wa: 21.506 GHz

 g: 215.06 MHz

T: 5e-08
nt: 25
dt: 1.9999999999999997e-09



In [72]:
H = Hamiltonian(capacity=config.capacity, cavity=cavity)

H.print_states()

States:
[0, [0, 0]]
[0, [0, 1]]
[0, [1, 0]]
[0, [1, 1]]
[1, [0, 0]]
[1, [0, 1]]
[1, [1, 0]]
[1, [1, 1]]
[2, [0, 0]]
[2, [0, 1]]
[2, [1, 0]]
[2, [1, 1]]



In [73]:
print(config.init_state)
print(config.init_state2)

w_0 = WaveFunction(states=H.states, init_state=config.init_state) - \
     WaveFunction(states=H.states, init_state=config.init_state2)
w_0.normalize()
w_0.print()

[0, [0, 0]]
[0, [1, 1]]
[0, [0, 0]] (0.7071067811865475+0j)
[0, [0, 1]] 0j
[0, [1, 0]] 0j
[0, [1, 1]] (-0.7071067811865475+0j)
[1, [0, 0]] 0j
[1, [0, 1]] 0j
[1, [1, 0]] 0j
[1, [1, 1]] 0j
[2, [0, 0]] 0j
[2, [0, 1]] 0j
[2, [1, 0]] 0j
[2, [1, 1]] 0j


In [74]:
run_w(w_0, H, dt=config.dt, nt=config.nt, config=config, fidelity_mode=False)

In [110]:
import plotly.graph_objs as go
import plotly.plotly as py
import numpy as np
import pandas as pd

import math
import csv

# BEGIN--------------------------------------------------- PLOTLY -----------------------------------------------------
import plotly

# plotly.tools.set_credentials_file(username="alexfmsu", api_key="g8ocp0PgQCY1a2WqBpyr")
plotly.tools.set_credentials_file(
    username="alexf-msu", api_key="VSOCzkhAhdKQDuV7eiYq")

# END----------------------------------------------------- PLOTLY -----------------------------------------------------
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

class PlotBuilder:
    def __init_(self):
        pass
    
    def set_title(self, title):
        self.title = title
        
    def set_width(self, width):
        self.width = width
    
    def set_height(self, height):
        self.height = height
        
    def set_size(self, width, height):
        self.set_width(width)
        self.set_height(height)
        
    def set_xticks(self, xticks):
        self.xticks = xticks
    
    def set_xaxis(self, xaxis):
        self.xaxis = xaxis

    def set_yaxis(self, yaxis):
        self.yaxis = yaxis
    
    def set_zrange(self, zrange):
        self.zrange = zrange
    
    def set_yscale(self, yscale):
        self.yscale = yscale

    def plot(self, z_csv, x_csv, y_csv, t_coeff=1, online=True, path=".", filename="wt2", to_file=""):
        print("Making plot...")
        # Z----------------------------------------------
        z_data = pd.read_csv(z_csv, header=None)
        # Z----------------------------------------------

        # X----------------------------------------------
        x = pd.read_csv(x_csv, keep_default_na=False)

        x.replace(r'\[(.+)\]', r'{\1≻', regex=True, inplace=True)


        x_header = list(x)[0]
        x_ticktext = list(x['x'])
        x_tickvals = list(x['vals'])

        for i in range(len(x_ticktext)):
            x_ticktext[i] = x_ticktext[i]
            x_ticktext[i] = str(x_ticktext[i])

        print('x_ticktext:', x_ticktext)
        print('x_tickvals:', x_tickvals)
        # X----------------------------------------------

        # Y----------------------------------------------
        y = pd.read_csv(y_csv, keep_default_na=False)
        y_header = list(y)[0]

        y_ticktext = list(y["y"])
        y_tickvals = list(y["vals"])
        y_tickvals = np.array(y_tickvals) / t_coeff
        print('y_ticktext:', y_ticktext)
        print('y_tickvals:', y_tickvals)
        # Y----------------------------------------------

        data = [
            go.Surface(
                showlegend=False,
                showscale=False,
                lighting=dict(diffuse=0.5, specular=.2, fresnel=0.2),
                z=z_data.values,
                colorscale="Portland",
            )
        ]

        scale = int(y_ticktext[-1])

        layout = go.Layout(
            title=self.title,
            titlefont=dict(
                family='Lato',

                size=20,
                color="#222"),
            margin=go.Margin(
                l=0,
                r=0,
                b=0,
                t=35,
                pad=50,
            ),
            xaxis=dict(
                ticks='outside',
                tickfont=dict(
                    size=200,
                ),
            ),

            yaxis=dict(
                # tickangle=45,

                title="y Axis",
                titlefont=dict(
                    family="Courier New, monospace",
                    # family='Old Standard TT, serif',
                    size=40,
                    # size=14,
                    color="#FFFFFF"),
                # autotick=False,
                # dtick=1,
                ticks='outside',
                tickfont=dict(
                    size=200,
                ),
            ),


            autosize=False,
#             width=1200,
    #         height=650,
            plot_bgcolor="#AAA",

            scene=go.Scene(
                camera=dict(
                    up=dict(x=0, y=0, z=1),
                    center=dict(x=0, y=0, z=0.2),
                    eye=dict(x=3.75, y=3.75, z=3.75)
                ),
                aspectratio={"x": 1, "y": self.yscale * y_ticktext[-1], "z": 1},
                xaxis={
                    "title": self.xaxis,
                    "showgrid": False,
                    "showline": False,
                    "tickvals": x_tickvals,
                    "ticktext": x_ticktext,
                    'titlefont': dict(
                        size=18,
                    ),
                    'tickfont': dict(
                        size=14,
                    ),
                    'autorange': True,
                },
                yaxis={
                    'autorange': True,

                    "title": self.yaxis+"\t\t\t\t.",
                    "ticktext": y_ticktext[::2],
                    "tickvals": y_tickvals[::2],
                    # "linecolor": "black",
                    "linewidth": 1,
                    'titlefont': dict(
                        size=18,
                    ),
                    'tickfont': dict(
                        size=14,
                    ),

                },
                zaxis={
                    'autorange': False,
                    "range": self.zrange,
                    "title": "prob.\t\t\t\t\t\t.",
                    "linewidth": 1,
                    'titlefont': dict(
                        size=18,
                    ),
                    'tickfont': dict(
                        size=14,
                    )
                },
            ),

            showlegend=False
        )

        fig = go.Figure(data=data, layout=layout)

        iplot(fig, filename=filename)

        return
# ---------------------------------------------------------------------------------------------------------------------


In [111]:
title = ""
    
plot_builder = PlotBuilder()
    
plot_builder.set_title(title)
plot_builder.set_xaxis('states')
plot_builder.set_yaxis("time, " + T_str_mark(config.T))
plot_builder.set_zrange([0,1])
plot_builder.set_yscale(0.1)


plot_builder.plot(
    z_csv=config.path + "/" + "z.csv",
    x_csv=config.path + "/" + "x.csv",
    y_csv=config.path + "/" + "t.csv",
    path=config.path,
    filename="Bipartite",
)


Making plot...
x_ticktext: ['', '', '', '', '', '', '', '', '', '', '', '']
x_tickvals: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
y_ticktext: [0.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0, 40.0, 45.0, 50.0]
y_tickvals: [ 0.   2.5  5.   7.5 10.  12.5 15.  17.5 20.  22.5 25. ]
